**Rule-Based Classification for Calculating Potential Customer Revenue**

**Business Problem**

A company wants to create new customer profiles (personas) based on certain characteristics of its existing customers using a level-based approach. The goal is to create segments according to these new customer definitions and estimate how much potential new customers could contribute to the company on average. For example, it is desired to estimate how much an average 25-year-old male iOS user from Turkey might contribute to the company.

**Data Set Story**

The Persona.csv dataset contains information about the prices of products sold by an international company and some demographic details of the users who purchased these products. The dataset is derived from records of each sales transaction. This means that the data is not aggregated, and a user with specific demographic characteristics may have made multiple purchases.

**Variables :**
* PRICE – Müşterinin harcama tutarı 
* SOURCE – Müşterinin bağlandığı cihaz türü 
* SEX – Müşterinin cinsiyeti 
* COUNTRY – Müşterinin ülkesi 
* AGE – Müşterinin yaşı

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/persona-data-set-0/persona.csv


In [2]:
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 500)
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
df = pd.read_csv("/kaggle/input/persona-data-set-0/persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [4]:
def check_df(dataframe, head=5):
    print("################ Shape ##############")
    print(dataframe.shape)
    print('################ Types ##############')
    print(dataframe.dtypes)
    print('################ Head ###############')
    print(dataframe.head(head))
    print('################ Tail ###############')
    print(dataframe.tail(head))
    print('################ NA ###############')
    print(dataframe.isnull().sum())
    print('################ Quantiles ###############')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

################ Shape ##############
(5000, 5)
################ Types ##############
PRICE       int64
SOURCE     object
SEX        object
COUNTRY    object
AGE         int64
dtype: object
################ Head ###############
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17
################ Tail ###############
      PRICE   SOURCE     SEX COUNTRY  AGE
4995     29  android  female     bra   31
4996     29  android  female     bra   31
4997     29  android  female     bra   31
4998     39  android  female     bra   31
4999     29  android  female     bra   31
################ NA ###############
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64
################ Quantiles ###############
        count     mean        std   min    0%    5%   50%   95%   99%  100%   max
PRICE  5000.0  34.132

Soru 2: Kaç unique SOURCE vardır? Frekansları nedir?

In [5]:
df["SOURCE"].nunique()

2

In [6]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

Soru 3: Kaç unique PRICE vardır?

In [7]:
df["PRICE"].nunique()

6

In [8]:
df["PRICE"].unique()

array([39, 49, 29, 19, 59,  9])

In [9]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [10]:
df["SEX"].nunique()

2

In [11]:
df["SEX"].unique()

array(['male', 'female'], dtype=object)

In [12]:
df["SEX"].value_counts()

SEX
female    2621
male      2379
Name: count, dtype: int64

In [13]:
df["COUNTRY"].nunique()

6

In [14]:
df["COUNTRY"].unique()

array(['bra', 'tur', 'usa', 'can', 'deu', 'fra'], dtype=object)

In [15]:
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

In [16]:
df["AGE"].nunique()

46

In [17]:
df["AGE"].unique()

array([17, 26, 23, 15, 19, 21, 16, 35, 42, 40, 49, 34, 18, 32, 25, 30, 20,
       22, 50, 37, 24, 31, 28, 66, 27, 47, 57, 52, 39, 59, 46, 36, 29, 51,
       55, 53, 33, 43, 38, 56, 54, 41, 44, 45, 65, 61])

In [18]:
df["AGE"].value_counts()

AGE
15    569
16    494
17    450
18    348
19    309
20    264
21    257
22    242
26    216
23    210
25    180
24    173
31    120
27    113
28    109
33     95
35     87
37     70
32     68
42     66
29     63
34     61
30     57
38     42
43     36
50     33
44     29
41     28
36     25
45     24
51     23
40     21
47     20
49     19
46     12
57      8
65      8
59      8
56      7
66      7
52      7
53      6
39      5
54      4
55      4
61      3
Name: count, dtype: int64

Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş?

In [19]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

Alternative Solution-2

In [20]:
df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

Alternative Solution-3

In [21]:
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


Soru 5: Hangi ülkeden kaçar tane satış olmuş?

In [22]:
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

Alternative Solution-2

In [23]:
# alternatif 2

df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

Alternative Solution-3

In [24]:
# alternatif 3

df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış?

In [25]:
df.groupby("COUNTRY")["PRICE"].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

Alternative Solution-2

In [26]:
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


Alternative Solution-3

In [27]:
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="sum")

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


Soru 7: SOURCE türlerine göre göre satış sayıları nedir?

In [28]:
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

Soru 8: Ülkelere göre PRICE ortalamaları nedir?

In [29]:
df.groupby('COUNTRY').agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?

In [30]:
df.groupby('SOURCE').agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?

In [31]:
df.groupby(["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

GÖREV 2: COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?

In [32]:
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

GÖREV 3: Çıktıyı PRICE'a göre sıralayınız.¶
Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
Çıktıyı agg_df olarak kaydediniz.

In [33]:
agg_df = df.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


GÖREV 4: Indekste yer alan isimleri değişken ismine çeviriniz.¶
Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
Bu isimleri değişken isimlerine çeviriniz.
İpucu: reset_index()
agg_df.reset_index(inplace=True)

In [34]:
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


GÖREV 5: AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.¶
Age sayısal değişkenini kategorik değişkene çeviriniz.
Aralıkları ikna edici olacağını düşündüğünüz şekilde oluşturunuz.
Örneğin: '0_18', '19_23', '24_30', '31_40', '41_70'

In [35]:
# AGE değişkeninin nerelerden bölüneceğini belirtelim:

bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

# Bölünen noktalara karşılık isimlendirmelerin ne olacağını ifade edelim:

mylabels = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())]

# age'i bölelim:

agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins, labels=mylabels)

agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


GÖREV 6: Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.¶
customers_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
Dikkat! list comp ile customers_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir.
Örneğin birden fazla şu ifadeden olabilir: USA_ANDROID_MALE_0_18. Bunları groupby'a alıp price ortalamalarını almak gerekmektedir.

In [36]:
agg_df.columns # değişken isimleri

Index(['COUNTRY', 'SOURCE', 'SEX', 'AGE', 'PRICE', 'AGE_CAT'], dtype='object')

In [37]:
# gözlem değerlerine erişmek için:

for row in agg_df.head(20).values:
    print(row)

# agg_df.values, dataframe'in tüm verilerini Numpy array'i olarak döndürüyor

['bra' 'android' 'male' 46 59.0 '41_66']
['usa' 'android' 'male' 36 59.0 '31_40']
['fra' 'android' 'female' 24 59.0 '24_30']
['usa' 'ios' 'male' 32 54.0 '31_40']
['deu' 'android' 'female' 36 49.0 '31_40']
['deu' 'android' 'female' 51 49.0 '41_66']
['deu' 'ios' 'male' 20 49.0 '19_23']
['usa' 'android' 'female' 47 49.0 '41_66']
['fra' 'ios' 'male' 20 49.0 '19_23']
['usa' 'ios' 'male' 59 46.5 '41_66']
['deu' 'android' 'male' 15 46.5 '0_18']
['bra' 'android' 'female' 30 45.666666666666664 '24_30']
['usa' 'android' 'female' 61 45.666666666666664 '41_66']
['tur' 'ios' 'male' 24 45.0 '24_30']
['deu' 'android' 'male' 34 45.0 '31_40']
['tur' 'ios' 'female' 18 44.0 '0_18']
['bra' 'android' 'female' 43 44.0 '41_66']
['bra' 'ios' 'female' 22 44.0 '19_23']
['tur' 'android' 'female' 32 43.0 '31_40']
['bra' 'android' 'female' 39 43.0 '31_40']


In [38]:
# COUNTRY, SOURCE, SEX ve AGE_CAT değişkenlerinin DEĞERLERİNİ yan yana koymak ve alt tireyle birleştirmek istiyoruz:

[row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]

# Veri setine ekleyelim:

agg_df["customers_level_based"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [39]:
# Gereksiz değişkenleri çıkaralım:

agg_df = agg_df[["customers_level_based", "PRICE"]]
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41_66,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0


In [40]:
# Birçok aynı segment olabilir, örneğin USA_ANDROID_MALE_0_18 segmentinden birçok sayıda olabilir
# Kontrol edelim:

agg_df["customers_level_based"].value_counts()

customers_level_based
BRA_ANDROID_MALE_24_30      7
USA_ANDROID_MALE_41_66      7
USA_IOS_FEMALE_24_30        7
BRA_ANDROID_FEMALE_24_30    7
USA_ANDROID_MALE_24_30      7
                           ..
TUR_ANDROID_MALE_41_66      1
CAN_ANDROID_MALE_19_23      1
TUR_IOS_MALE_31_40          1
TUR_IOS_MALE_24_30          1
CAN_ANDROID_FEMALE_24_30    1
Name: count, Length: 109, dtype: int64

In [41]:
# Bu sebeple segmentlere göre groupby yaptıktan sonra PRICE ortalamalarını almalı ve segmentleri tekilleştirmeliyiz:

agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})

In [42]:
# customers_level_based index'te yer alıyormu, kontrol edelim:

agg_df.head()

,PRICE
customers_level_based,
BRA_ANDROID_FEMALE_0_18,35.645303
BRA_ANDROID_FEMALE_19_23,34.077340
BRA_ANDROID_FEMALE_24_30,33.863946
BRA_ANDROID_FEMALE_31_40,34.898326
BRA_ANDROID_FEMALE_41_66,36.737179


In [43]:
# Değişkene çevirelim:

agg_df = agg_df.reset_index()
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


In [44]:
# Heri bir persona'nın 1 tane olması gerekiyor. Kontrol edelim:

agg_df["customers_level_based"].value_counts()

customers_level_based
BRA_ANDROID_FEMALE_0_18    1
TUR_ANDROID_FEMALE_0_18    1
TUR_IOS_FEMALE_19_23       1
TUR_IOS_FEMALE_0_18        1
TUR_ANDROID_MALE_41_66     1
                          ..
CAN_IOS_MALE_41_66         1
CAN_IOS_MALE_31_40         1
CAN_IOS_MALE_24_30         1
CAN_IOS_MALE_0_18          1
USA_IOS_MALE_41_66         1
Name: count, Length: 109, dtype: int64

In [45]:
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


GÖREV 7: Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.¶
1. PRICE'a göre segmentlere ayırın.
2. Segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyin.
3. Segmentleri betimleyin.

In [46]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head(10)

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A
5,BRA_ANDROID_MALE_0_18,34.805861,B
6,BRA_ANDROID_MALE_19_23,31.673243,D
7,BRA_ANDROID_MALE_24_30,33.413919,C
8,BRA_ANDROID_MALE_31_40,34.327381,B
9,BRA_ANDROID_MALE_41_66,40.041667,A


In [47]:
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

,PRICE
SEGMENT,
D,29.206780
C,33.509674
B,34.999645
A,38.691234


In [48]:
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A


GÖREV 8: Yeni gelen müşterileri sınıflandırınız ne kadar gelir getirebileceğini tahmin ediniz.¶
Soru 1. 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?

In [49]:
new_user = "TUR_ANDROID_FEMALE_31_40"

agg_df[agg_df["customers_level_based"] == new_user]

,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


Soru 2. 35 yaşında IOS kullanan bir Fransız kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?

In [50]:
new_user = "FRA_IOS_FEMALE_31_40"

agg_df[agg_df["customers_level_based"] == new_user]

,customers_level_based,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C
